In [2]:
!pip install flask flask-cors spacy flask_ngrok


In [3]:
!python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 17.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
!pip install scikit-learn


In [7]:
!pip install pyngrok

In [8]:
from pyngrok import ngrok

# Replace below with your actual token
ngrok.set_auth_token("2z4kW9QCMh8NhfG7kWaijKmC8h2_4xYiLo3UPtEfQ3zjYCzPb")



In [10]:
import spacy
from sklearn.cluster import KMeans
import numpy as np
from collections import defaultdict
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

nlp = spacy.load("en_core_web_md")

# ---- STEP 1: Compound Phrase Extraction ----
def extract_compound_phrases(doc):
    compound_map = {}
    for token in doc:
        if token.dep_ == "compound":
            head = token.head.i
            compound_map.setdefault(head, []).append(token)

    phrases = set()
    used_token_ids = set()

    for head_id, compound_tokens in compound_map.items():
        head_token = doc[head_id]
        full_phrase = " ".join([t.text for t in sorted(compound_tokens, key=lambda t: t.i)] + [head_token.text])
        phrases.add(full_phrase.lower())
        used_token_ids.update([t.i for t in compound_tokens] + [head_id])

    return phrases, used_token_ids

# ---- STEP 2: Extract Relations ----
def extract_relations(doc):
    relations = []
    for token in doc:
        if token.dep_ in ("ROOT", "relcl", "xcomp") and token.pos_ == "VERB":
            subj = None
            obj = None
            for child in token.children:
                if child.dep_ in ("nsubj", "nsubjpass"):
                    subj = child
                elif child.dep_ in ("dobj", "pobj", "attr", "xcomp"):
                    obj = child
            if subj and obj:
                relations.append({
                    "source": subj.text.lower(),
                    "target": obj.text.lower(),
                    "relation": token.lemma_.lower()
                })
    return relations

# ---- STEP 3: Keyword Extraction ----
def extract_keywords(text):
    doc = nlp(text)
    keywords = set()

    compound_phrases, used_ids = extract_compound_phrases(doc)
    keywords.update(compound_phrases)

    for chunk in doc.noun_chunks:
        if not set([t.i for t in chunk]).issubset(used_ids):
            keywords.add(chunk.text.strip().lower())

    for ent in doc.ents:
        keywords.add(ent.text.strip().lower())

    for token in doc:
        if token.pos_ in ("NOUN", "PROPN") and token.i not in used_ids and len(token.text.strip()) > 2:
            keywords.add(token.text.strip().lower())

    return list(set([kw for kw in keywords if len(kw) > 2]))

# ---- STEP 4: Embedding Generator ----
def get_embedding(text):
    doc = nlp(text)
    vecs = [t.vector for t in doc if t.has_vector]
    return np.mean(vecs, axis=0) if vecs else np.zeros(nlp.vocab.vectors_length)

# ---- STEP 5: Smart Cluster Labels ----
def infer_cluster_label(keywords):
    for kw in keywords:
        if any(tag in kw for tag in ["define", "definition", "is", "refers to"]):
            return "Definition"
        elif any(tag in kw for tag in ["use", "used", "purpose", "goal"]):
            return "Application"
        elif any(tag in kw for tag in ["method", "technique", "type", "model"]):
            return "Technique"
    return keywords[0]  # fallback

# ---- STEP 6: Clustering ----
def cluster_keywords(keywords, n_clusters=5):
    embeddings = np.array([get_embedding(kw) for kw in keywords])
    valid_idx = np.array([not np.all(v == 0) for v in embeddings])
    keywords = [kw for i, kw in enumerate(keywords) if valid_idx[i]]
    embeddings = embeddings[valid_idx]

    if len(embeddings) < n_clusters:
        n_clusters = max(1, len(embeddings))

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(embeddings)

    clustered = defaultdict(list)
    for kw, label in zip(keywords, labels):
        clustered[label].append(kw)

    result = []
    for label, group in clustered.items():
        cluster_label = infer_cluster_label(group)
        result.append({
            "cluster_id": label,
            "cluster_label": cluster_label,
            "keywords": group
        })

    return result

# ---- STEP 7: Mind Map Node Generator ----
def extract_mindmap_nodes(text):
    doc = nlp(text)
    keywords = extract_keywords(text)
    clusters = cluster_keywords(keywords, n_clusters=5)
    relations = extract_relations(doc)

    node_map = {}
    nodes = []

    for cluster in clusters:
        cluster_node = {
            "id": f"cluster_{cluster['cluster_id']}",
            "label": cluster["cluster_label"],
            "type": "cluster",
            "children": []
        }

        for kw in cluster["keywords"]:
            kw_id = f"node_{kw.replace(' ', '_')}"
            cluster_node["children"].append({
                "id": kw_id,
                "label": kw,
                "type": "keyword"
            })
            node_map[kw.lower()] = kw_id

        node_map[cluster["cluster_label"].lower()] = cluster_node["id"]
        nodes.append(cluster_node)

    relation_edges = []
    for rel in relations:
        src = node_map.get(rel["source"])
        tgt = node_map.get(rel["target"])
        if src and tgt:
            relation_edges.append({
                "source": src,
                "target": tgt,
                "label": rel["relation"]
            })

    return {
        "nodes": nodes,
        "relations": relation_edges
    }

In [15]:
from flask import Flask, request, jsonify
from flask_cors import CORS


from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

@app.route("/")
def home():
    return "ThinkSpace Backend is Running 🧠"

@app.route("/process", methods=["POST"])
def process_text():
    data = request.get_json()
    text = data.get("text", "")
    if not text.strip():
        return jsonify({"error": "Empty text provided"}), 400

    try:
        result = extract_mindmap_nodes(text)
        return jsonify(result)  # ✅ Final correct return
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    port = 5000
    public_url = ngrok.connect(port).public_url
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
    app.run(port=port)


 * ngrok tunnel "https://3ad7-34-16-252-90.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Jun/2025 14:41:57] "OPTIONS /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jun/2025 14:41:58] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jun/2025 14:50:41] "OPTIONS /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jun/2025 14:50:42] "POST /process HTTP/1.1" 200 -
